In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-oct-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-oct-2022/train_9.csv
/kaggle/input/tabular-playground-series-oct-2022/train_3.csv
/kaggle/input/tabular-playground-series-oct-2022/train_4.csv
/kaggle/input/tabular-playground-series-oct-2022/test_dtypes.csv
/kaggle/input/tabular-playground-series-oct-2022/train_8.csv
/kaggle/input/tabular-playground-series-oct-2022/train_dtypes.csv
/kaggle/input/tabular-playground-series-oct-2022/train_5.csv
/kaggle/input/tabular-playground-series-oct-2022/train_2.csv
/kaggle/input/tabular-playground-series-oct-2022/test.csv
/kaggle/input/tabular-playground-series-oct-2022/train_0.csv
/kaggle/input/tabular-playground-series-oct-2022/train_7.csv
/kaggle/input/tabular-playground-series-oct-2022/train_1.csv
/kaggle/input/tabular-playground-series-oct-2022/train_6.csv


# Intro
1. **The kernel refers to CHAZZER's work(https://www.kaggle.com/code/chazzer/rocket-league-xgboost-feat-engineering-cv)**
2. The kernel focuses on feature engineering.   

# Modules Input

In [36]:
import os  # file navigation
import gc  # garbage collection

# visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots

from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier

# check if GPU is available
from tensorflow.config import list_physical_devices


In [37]:
GPU = list_physical_devices('GPU') != []
N_ESTIMATORS = 2000
MAX_DEPTH = 8
LEARNING_RATE = 0.01
FOLDS = 5

DEBUG = False
SAMPLE = 0.2
SEED = 42

# Data Input

In [38]:
train0 = pd.read_csv("../input/tabular-playground-series-oct-2022/train_0.csv")

# Basic info of data

In [39]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149381 entries, 0 to 2149380
Data columns (total 61 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   game_num                     int64  
 1   event_id                     int64  
 2   event_time                   float64
 3   ball_pos_x                   float64
 4   ball_pos_y                   float64
 5   ball_pos_z                   float64
 6   ball_vel_x                   float64
 7   ball_vel_y                   float64
 8   ball_vel_z                   float64
 9   p0_pos_x                     float64
 10  p0_pos_y                     float64
 11  p0_pos_z                     float64
 12  p0_vel_x                     float64
 13  p0_vel_y                     float64
 14  p0_vel_z                     float64
 15  p0_boost                     float64
 16  p1_pos_x                     float64
 17  p1_pos_y                     float64
 18  p1_pos_z                     float64
 19  

In [40]:
train0.describe()

,game_num,event_id,event_time,ball_pos_x,ball_pos_y,ball_pos_z,ball_vel_x,ball_vel_y,ball_vel_z,p0_pos_x,...,p5_boost,boost0_timer,boost1_timer,boost2_timer,boost3_timer,boost4_timer,boost5_timer,player_scoring_next,team_A_scoring_within_10sec,team_B_scoring_within_10sec
count,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.130390e+06,...,2.130318e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06,2.149381e+06
mean,3.678573e+02,3.678601e+05,-6.615076e+01,-1.711843e-01,-8.701548e-03,1.566230e+01,-3.763223e-03,2.237860e-02,1.148253e-01,-8.151923e-02,...,4.577625e+01,-2.727069e+00,-2.754303e+00,-3.226214e+00,-3.220364e+00,-2.751841e+00,-2.718325e+00,1.604489e+00,5.831074e-02,5.575280e-02
std,2.125500e+02,2.125501e+05,6.682579e+01,4.909460e+01,6.008009e+01,1.044591e+01,2.316391e+01,2.806578e+01,1.228911e+01,4.550489e+01,...,3.410338e+01,3.301398e+00,3.306345e+00,3.364020e+00,3.363124e+00,3.306341e+00,3.300695e+00,2.080762e+00,2.343302e-01,2.294438e-01
min,1.000000e+00,1.002000e+03,-7.205285e+02,-8.079200e+01,-1.043092e+02,1.296000e+00,-9.920059e+01,-8.797200e+01,-6.299300e+01,-8.189000e+01,...,0.000000e+00,-9.984000e+00,-9.984000e+00,-9.970000e+00,-9.984000e+00,-9.970000e+00,-9.970000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,1.830000e+02,1.830040e+05,-9.148513e+01,-4.498640e+01,-5.354000e+01,6.395000e+00,-1.558660e+01,-2.184260e+01,-7.648400e+00,-3.730970e+01,...,1.294000e+01,-5.477000e+00,-5.527000e+00,-6.200000e+00,-6.195000e+00,-5.523000e+00,-5.465000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.650000e+02,3.650040e+05,-4.588233e+01,0.000000e+00,0.000000e+00,1.412580e+01,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,...,4.275000e+01,-6.904000e-01,-7.900000e-01,-2.193000e+00,-2.186000e+00,-7.803000e-01,-6.540000e-01,1.000000e+00,0.000000e+00,0.000000e+00
75%,5.530000e+02,5.530020e+05,-1.925391e+01,4.450460e+01,5.377380e+01,2.366960e+01,1.563160e+01,2.181040e+01,7.640200e+00,3.711940e+01,...,7.694000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00
max,7.370000e+02,7.370050e+05,-1.714620e-02,8.069160e+01,1.043092e+02,3.943720e+01,9.838720e+01,1.011442e+02,6.794280e+01,8.190180e+01,...,1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00


In [41]:
train0.shape

(2149381, 61)

In [42]:
train0.columns

Index(['game_num', 'event_id', 'event_time', 'ball_pos_x', 'ball_pos_y',
       'ball_pos_z', 'ball_vel_x', 'ball_vel_y', 'ball_vel_z', 'p0_pos_x',
       'p0_pos_y', 'p0_pos_z', 'p0_vel_x', 'p0_vel_y', 'p0_vel_z', 'p0_boost',
       'p1_pos_x', 'p1_pos_y', 'p1_pos_z', 'p1_vel_x', 'p1_vel_y', 'p1_vel_z',
       'p1_boost', 'p2_pos_x', 'p2_pos_y', 'p2_pos_z', 'p2_vel_x', 'p2_vel_y',
       'p2_vel_z', 'p2_boost', 'p3_pos_x', 'p3_pos_y', 'p3_pos_z', 'p3_vel_x',
       'p3_vel_y', 'p3_vel_z', 'p3_boost', 'p4_pos_x', 'p4_pos_y', 'p4_pos_z',
       'p4_vel_x', 'p4_vel_y', 'p4_vel_z', 'p4_boost', 'p5_pos_x', 'p5_pos_y',
       'p5_pos_z', 'p5_vel_x', 'p5_vel_y', 'p5_vel_z', 'p5_boost',
       'boost0_timer', 'boost1_timer', 'boost2_timer', 'boost3_timer',
       'boost4_timer', 'boost5_timer', 'player_scoring_next',
       'team_scoring_next', 'team_A_scoring_within_10sec',
       'team_B_scoring_within_10sec'],
      dtype='object')

# Feature Engineering

## Euclidian Norm

In [43]:
def euclidian_norm(x):
    return np.linalg.norm(x, axis=1)

In [44]:
vel_groups = {f'{ele}_vel': [f'{ele}_vel_x', f'{ele}_vel_y', f'{ele}_vel_z'] for ele in ['ball'] + [f'p{i}' for i in range(6)]}
vel_groups

{'ball_vel': ['ball_vel_x', 'ball_vel_y', 'ball_vel_z'],
 'p0_vel': ['p0_vel_x', 'p0_vel_y', 'p0_vel_z'],
 'p1_vel': ['p1_vel_x', 'p1_vel_y', 'p1_vel_z'],
 'p2_vel': ['p2_vel_x', 'p2_vel_y', 'p2_vel_z'],
 'p3_vel': ['p3_vel_x', 'p3_vel_y', 'p3_vel_z'],
 'p4_vel': ['p4_vel_x', 'p4_vel_y', 'p4_vel_z'],
 'p5_vel': ['p5_vel_x', 'p5_vel_y', 'p5_vel_z']}

In [45]:
pos_groups = {f'{ele}_pos': [f'{ele}_pos_x', f'{ele}_pos_y', f'{ele}_pos_z'] for ele in ['ball'] + [f'p{i}' for i in range(6)]}
pos_groups

{'ball_pos': ['ball_pos_x', 'ball_pos_y', 'ball_pos_z'],
 'p0_pos': ['p0_pos_x', 'p0_pos_y', 'p0_pos_z'],
 'p1_pos': ['p1_pos_x', 'p1_pos_y', 'p1_pos_z'],
 'p2_pos': ['p2_pos_x', 'p2_pos_y', 'p2_pos_z'],
 'p3_pos': ['p3_pos_x', 'p3_pos_y', 'p3_pos_z'],
 'p4_pos': ['p4_pos_x', 'p4_pos_y', 'p4_pos_z'],
 'p5_pos': ['p5_pos_x', 'p5_pos_y', 'p5_pos_z']}

In [46]:
# velocity magnitude
for col, vec in vel_groups.items():
    train0[col] = euclidian_norm(train0[vec])
    

In [47]:
# distance to ball
for col, feat in pos_groups.items():
    train0[col+'_ball_distance'] = euclidian_norm(train0[feat].values - train0[pos_groups['ball_pos']].values)


## Cleaning

In [48]:
cols_drop = ['game_num', 'event_id', 'event_time', 'player_scoring_next', 'team_scoring_next', 'ball_pos_ball_distance']
train0 = train0.drop(columns=cols_drop)

In [49]:
train0.columns

Index(['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x', 'ball_vel_y',
       'ball_vel_z', 'p0_pos_x', 'p0_pos_y', 'p0_pos_z', 'p0_vel_x',
       'p0_vel_y', 'p0_vel_z', 'p0_boost', 'p1_pos_x', 'p1_pos_y', 'p1_pos_z',
       'p1_vel_x', 'p1_vel_y', 'p1_vel_z', 'p1_boost', 'p2_pos_x', 'p2_pos_y',
       'p2_pos_z', 'p2_vel_x', 'p2_vel_y', 'p2_vel_z', 'p2_boost', 'p3_pos_x',
       'p3_pos_y', 'p3_pos_z', 'p3_vel_x', 'p3_vel_y', 'p3_vel_z', 'p3_boost',
       'p4_pos_x', 'p4_pos_y', 'p4_pos_z', 'p4_vel_x', 'p4_vel_y', 'p4_vel_z',
       'p4_boost', 'p5_pos_x', 'p5_pos_y', 'p5_pos_z', 'p5_vel_x', 'p5_vel_y',
       'p5_vel_z', 'p5_boost', 'boost0_timer', 'boost1_timer', 'boost2_timer',
       'boost3_timer', 'boost4_timer', 'boost5_timer',
       'team_A_scoring_within_10sec', 'team_B_scoring_within_10sec',
       'ball_vel', 'p0_vel', 'p1_vel', 'p2_vel', 'p3_vel', 'p4_vel', 'p5_vel',
       'p0_pos_ball_distance', 'p1_pos_ball_distance', 'p2_pos_ball_distance',
       'p3_pos_ball

In [50]:
train0 = train0.dropna(axis=0)

In [51]:
train0.shape

(2039435, 69)

# Building Models

In [52]:
le_a = LabelEncoder()

In [53]:
model_a = XGBClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=LEARNING_RATE, objective='binary:logistic', tree_method='gpu_hist' if GPU else 'hist')

In [56]:
cv_a = cross_validate(model_a, X=train0.drop(columns=['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec']).values, 
                      y=le_a.fit_transform(train0['team_A_scoring_within_10sec'].values),
                      scoring='neg_log_loss',
                      cv=FOLDS,
                      verbose=2,
                      return_estimator=True
                     )

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=18.2min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.2min remaining:    0.0s


[CV] END .................................................... total time=18.2min
[CV] END .................................................... total time=18.2min
[CV] END .................................................... total time=18.0min
[CV] END .................................................... total time=18.7min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 91.3min finished


In [57]:
le_b = LabelEncoder()

In [58]:
model_b = XGBClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=LEARNING_RATE, objective='binary:logistic', tree_method='gpu_hist' if GPU else 'hist')

In [60]:
cv_b = cross_validate(model_b, 
                      X=train0.drop(columns=['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec']).values,
                      y=le_b.fit_transform(train0['team_B_scoring_within_10sec'].values),
                      cv=FOLDS,
                      verbose=2,
                      return_estimator=True
                     )

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=18.5min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.5min remaining:    0.0s


[CV] END .................................................... total time=18.3min
[CV] END .................................................... total time=18.2min
[CV] END .................................................... total time=18.7min
[CV] END .................................................... total time=18.1min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 91.9min finished
